In [1]:
import numpy as np
from sklearn import preprocessing 

## Extract the data from the csv

In [2]:
raw_csv_data = np.loadtxt('Audiobooks_data.csv', delimiter=',')

unscaled_inputs_all = raw_csv_data[:, 1:-1]
targets_all = raw_csv_data[:,-1]

## Balance/Scaling the dataset

### 1. We will count the number of targets that are 1s
### 2. We will keep as many 0s and 1s (we will delete the others)

In [3]:
# if we sum all the targets we will get the number of targets that are 1s
num_one_targets = int(np.sum(targets_all))
zero_targets_counter = 0
indices_to_remove = []

# The shape of targets_all on axis = 0, is basically the length of the vector
for i in range(targets_all.shape[0]):
    if targets_all[i] == 0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)

In [4]:
# np.delete(array, obj to delete, axis) is a method that deletes an object along an axis
unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis=0)
targets_equal_priors = np.delete(targets_all, indices_to_remove, axis=0)

In [5]:
targets_all.shape

(14084,)

In [6]:
unscaled_inputs_all.shape

(14084, 10)

In [16]:
unscaled_inputs_all[0]

array([1.6200e+03, 1.6200e+03, 1.9730e+01, 1.9730e+01, 1.0000e+00,
       1.0000e+01, 9.9000e-01, 1.6038e+03, 5.0000e+00, 9.2000e+01])

## Standardize the inputs

In [7]:
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)

## Shuffle the data

In [8]:
# shuffle the inputs and the targets. We keep the same information but in a random order
# Since we will be batching, so we must shuffle the data

# np.arange([start], stop) is a method that returns a evenly spaced values within a given interval
    # is the same as arange
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

In [9]:
scaled_inputs.shape

(4474, 10)

In [10]:
shuffled_indices

array([2868, 1807, 3747, ..., 2779,  725, 2513])

In [11]:
a = scaled_inputs[[0, 1]]
a

array([[ 2.10533868e-01, -1.88885170e-01,  1.97823887e+00,
         1.43819814e+00,  2.10780037e+00,  1.49110879e+00,
         4.24286158e+00,  4.80955413e+00,  1.18382842e+01,
         9.41504310e-02],
       [ 1.27894497e+00,  4.16467444e-01, -3.90824747e-01,
        -5.08367103e-01, -4.74428231e-01,  1.42655343e-03,
        -4.26477846e-01, -4.15699224e-01, -2.01834807e-01,
        -8.02558517e-01]])

## Split the dataset into train, validation, and test

In [12]:
# 80-10-10 split for train,validation, and test
samples_count = shuffled_inputs.shape[0]

train_samples_count = int(0.8*samples_count)
validation_samples_count = int(0.1*samples_count)
test_samples_count = samples_count - train_samples_count - validation_samples_count

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count: train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count: train_samples_count+validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]

In [13]:
# check if we have balanced the dataset
# the totla number of samples, and the proportion of ones as a part of the total
print(np.sum(train_targets), train_samples_count, np.sum(train_targets)/train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets)/validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets)/test_samples_count)

1798.0 3579 0.502374965074043
225.0 447 0.5033557046979866
214.0 448 0.47767857142857145


## Save the three datasets in *.npz

In [14]:
np.savez('Audiobooks_data_train_main', inputs=train_inputs, targets=train_targets)
np.savez('Audiobooks_data_validation_main', inputs=validation_inputs, targets=validation_targets)
np.savez('Audiobooks_data_test_main', inputs=test_inputs, targets=test_targets)